In [8]:
import torch

weight = torch.ones(1, 1, 2, 3, 3) # 1, out_channels, in_channels, kernel_size, kernel_size
style = torch.range(1, 4).reshape(2, 1, 2, 1, 1)
R = weight * style
R

/tmp/ipykernel_74580/1074074487.py:4: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  style = torch.range(1, 4).reshape(2, 1, 2, 1, 1)


tensor([[[[[1., 1., 1.],
           [1., 1., 1.],
           [1., 1., 1.]],

          [[2., 2., 2.],
           [2., 2., 2.],
           [2., 2., 2.]]]],



        [[[[3., 3., 3.],
           [3., 3., 3.],
           [3., 3., 3.]],

          [[4., 4., 4.],
           [4., 4., 4.],
           [4., 4., 4.]]]]])

In [4]:
s = s.replace('\n', ' ')

In [5]:
s

'In this paper, we will focus on an efficient deep neural  network architecture for computer vision, codenamed In ception, which derives its name from the Network in net work paper by Lin et al [12] in conjunction with the famous  “we need to go deeper” internet meme [1]. In our case, the  word “deep” is used in two different meanings: first of all,  in the sense that we introduce a new level of organization  in the form of the “Inception module” and also in the more  direct sense of increased network depth. In general, one can  view the Inception model as a logical culmination of [12]  while taking inspiration and guidance from the theoretical  work by Arora et al [2]. The benefits of the architecture are  experimentally verified on the ILSVRC 2014 classification  and detection challenges, where it significantly outperforms  the current state of the art.'